In [1]:
!pip install marshmallow
from marshmallow import Schema, fields, post_load, validate , ValidationError, validates
from pprint import pprint


     |████████████████████████████████| 51kB 1.7MB/s 


In [2]:
class User(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __repr__(self):
        return f'I am {self.name} and my age is {self.age}'

class UserSchema(Schema):
    name = fields.String()
    age = fields.Integer()

    @post_load
    def make(self, data, **kwargs):
        return User(**data)



In [3]:
data = {
    'name': 'Chris',
    'age': 32
}

schema = UserSchema()
user = schema.load(data)
print(user)


I am Chris and my age is 32


# JSON TO MULTIPLE OBJECTS

In [4]:
data = [{
    'name':'Alice',
    'age': 20

},{
    'name':'Bob',
    'age':25
},{
    'name':'Harry',
    'age':21
}]

schema =UserSchema()
users = schema.load(data,many=True)
pprint(users)


[I am Alice and my age is 20,
 I am Bob and my age is 25,
 I am Harry and my age is 21]


# Selialization

In [5]:
dict = schema.dump(users, many=True)
pprint(dict)

[{'age': 20, 'name': 'Alice'},
 {'age': 25, 'name': 'Bob'},
 {'age': 21, 'name': 'Harry'}]


# Validation

In [6]:

data = [{
    'name':'Ram',
    'age': 20

},{
    'name':'Shyam',
    'age':'hundred'
},{
    'name':'Harry',
    'age':21
}]

try:
    schema = UserSchema()
    users = schema.load(data, many=True)
except ValidationError as e:
    print(f'Error Msg:{e.messages}')
    print(f'Valid Data:{e.valid_data}')




Error Msg:{1: {'age': ['Not a valid integer.']}}
Valid Data:[{'age': 20, 'name': 'Ram'}, {'name': 'Shyam'}, {'age': 21, 'name': 'Harry'}]


# Advance Validation

In [7]:
class User(object):
    def __init__(self,name, age,gender):
        self.name = name
        self.age = age
        self.gender = gender
        
    def __repr__(self):
        return f'I am {self.name}, my age is {self.age} and my gender is {self.gender}'
    
class UserSchema(Schema):
    name = fields.String(validate =validate.Length(min = 1))
    age = fields.Integer(validate= validate.Range(min = 18, max = None))
    gender = fields. String(validate = validate.OneOf(['F','M','Other']))

In [8]:
data = {
    'name': 1,
    'age': 21,
    'gender': 'A'
}

try:
    UserSchema().load(data)
except ValidationError as e:
    pprint(e.messages)

{'gender': ['Must be one of: F, M, Other.'], 'name': ['Not a valid string.']}


# Customised Validate Functions

In [9]:
def validate_age(age):
    if age < 18:
        raise ValidationError('You mst be above 18 to proceed further')

class UserSchema(Schema):
    name = fields.String(validate=validate.Length(min=1))
    age = fields.Integer(validate=validate_age)
    gender = fields.String(validate=validate.OneOf(['F','M','Other']))
    

data = {
    'name': 'Harry',
    'age': 3,
    'gender': 'M'
}

try:
    user = UserSchema().load(data)
except ValidationError as e:
    pprint(e.messages)

{'age': ['You mst be above 18 to proceed further']}


# OR

In [10]:
class UserSchema(Schema):
    name = fields.String()
    age = fields.Integer()
    gender = fields.String()
    
    @validates('age')
    def validate_age(self, age):
        if age < 18:
            raise ValidationError(' You must be above 18 to procced further')

In [11]:
data = {
    'name': 'Harry',
    'age': 17,
    'gender': 'M'
}

try:
    user = UserSchema().load(data)
except ValidationError as e:
    pprint(e.messages)

{'age': [' You must be above 18 to procced further']}


# Required Fields

In [12]:
class UserSchema(Schema):
    name = fields.String(required=True, error_messages={'required':'Please enter your Name.'})
    age = fields.Integer(required=True, error_messages={'required':'Please enter your Age'})
    email = fields.Email()

data_no_email = {
    'name': 'Harry',
    'age': 21
}

try:
    user = UserSchema().load(data_no_email)
except ValidationError as e:
    pprint(e.messages)

print(user)

{'age': 21, 'name': 'Harry'}


In [14]:
data_no_email_age = {
    'email' : 'abc@gmail.com'
}

try:
    user = UserSchema().load(data_no_email_age)
except ValidationError as e:
    print(f'Error message: {e.messages}')
    print(f'Valid Data:{e.valid_data}')

Error message: {'age': ['Please enter your Age'], 'name': ['Please enter your Name.']}
Valid Data:{'email': 'abc@gmail.com'}
